In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import csv

# 웹 드라이버 설정
driver = webdriver.Chrome()  # 드라이버 경로
wait = WebDriverWait(driver, 20)

# 구글 검색 페이지 URL
base_url = 'https://www.google.com/search?q='
search_query = input('검색어를 입력하세요: ')
url = base_url + search_query

driver.get(url)

검색어를 입력하세요: 돈비어천가


In [6]:
# CSV 파일에 데이터를 저장하는 함수
def save_data_to_csv(data_list, output_file):
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['검색어', '내용'])
        for data in data_list:
            writer.writerow([data['search_query'], data['content']])

data_list = []

In [7]:
# 페이지 수 반복
for page_num in range(5):  # 크롤링할 페이지 수 (5페이지 까지)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    results = soup.select('.tF2Cxc')  # 변경된 CSS 선택자(이건 페이지마다 다름)

    for result in results:
        link = result.select_one('a')
        if link:
            link_address = link['href']
            driver.execute_script(f"window.open('{link_address}');")  # 새 창에서 링크 열기
            driver.switch_to.window(driver.window_handles[1])  # 새로 열린 창으로 전환

            try:
                wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))  # body가 로드될 때까지 대기
                page_html = driver.page_source
                page_soup = BeautifulSoup(page_html, 'html.parser')
                content = page_soup.find('body').text.strip()  # body의 내용 추출
                data = {
                    'search_query': search_query,
                    'content': content
                }
                data_list.append(data)
            except (TimeoutException, NoSuchElementException):
                print(f"링크 {link_address}에서 내용을 가져오지 못했습니다.")

            driver.close()  # 현재 창 닫기
            driver.switch_to.window(driver.window_handles[0])  # 원래 창으로 전환

    try:
        # 클릭할 다음 버튼 찾기
        next_button = driver.find_element(By.ID, 'pnnext')
    except NoSuchElementException:
        print("더 이상 다음 페이지가 없습니다.")
        break

    # 다음 버튼 클릭
    next_button.click()

# 웹 드라이버 종료
driver.quit()

if not data_list:
    print("내용을 찾을 수 없습니다.")
else:
    # 데이터를 CSV 파일에 저장
    output_file = f'google_search_contents_{search_query}.csv'
    save_data_to_csv(data_list, output_file)
    print(f'내용이 {output_file}에 저장되었습니다.')

내용이 google_search_contents_돈비어천가.csv에 저장되었습니다.


In [7]:
import pandas as pd
from konlpy.tag import Hannanum
df = pd.read_csv('google_search_contents_돈비어천가.csv', encoding='utf-8')
df

,검색어,내용
0,돈비어천가,언어를 선택해주세요.한국어ENG中文日本語식신 대한민국 NO.1 맛집 서비스오늘 뭐 ...
1,돈비어천가,언어 선택네팔어독일어러시아어몽골어미얀마어(버마어)베트남어스페인어아랍어영어인도네시아어...
2,돈비어천가,모바일보기\n|\n 내위치설정\n|\n 장바구니\n|\n 로그인 \n\t\t\...
3,돈비어천가,맛집 평가는 모바일 앱에서만 할 수 있어요.\n\n\n\n\n\n\n\n\n\n\n...
4,돈비어천가,메뉴안내\n음식점\n축제\n숙박\n레저\n\n© 2023 인천 여기유\n\n \n\...
5,돈비어천가,생생정보통 맛집오늘 태능갈비 돈비어천가 49년 전통 양념목살갈비 수제 돼지 왕갈비 ...
6,돈비어천가,본문 바로가기\n\n\n\n블로그\n\n\n\n\n\n카테고리 이동\n\n\n\n\...
7,돈비어천가,TOP 여행지내 예약APP돈비어천가 1월미도 주변출처 www.instagram.co...
8,돈비어천가,Login\n\n\n\n\n\n\n\n\nHome\nTags\nGuestbook\n...
9,돈비어천가,NaN


In [29]:
hannanum = Hannanum()
text_list = []
string_series = df['내용'].apply(str)
string_series

0     언어를 선택해주세요.한국어ENG中文日本語식신 대한민국 NO.1 맛집 서비스오늘 뭐 ...
1     언어 선택네팔어독일어러시아어몽골어미얀마어(버마어)베트남어스페인어아랍어영어인도네시아어...
2     모바일보기\n|\n 내위치설정\n|\n 장바구니\n|\n 로그인    \n\t\t\...
3     맛집 평가는 모바일 앱에서만 할 수 있어요.\n\n\n\n\n\n\n\n\n\n\n...
4     메뉴안내\n음식점\n축제\n숙박\n레저\n\n© 2023 인천 여기유\n\n \n\...
5     생생정보통 맛집오늘 태능갈비 돈비어천가 49년 전통 양념목살갈비 수제 돼지 왕갈비 ...
6     본문 바로가기\n\n\n\n블로그\n\n\n\n\n\n카테고리 이동\n\n\n\n\...
7     TOP 여행지내 예약APP돈비어천가 1월미도 주변출처 www.instagram.co...
8     Login\n\n\n\n\n\n\n\n\nHome\nTags\nGuestbook\n...
9                                                   nan
10    북마크\n\n\n\n\n\n\n\n\n\n검색어 필수\n\n\n검색\n\n\n\n\...
11    사진 더보기\n                    \n\n\n\n\n\n\n\n\n...
12    언어 선택네팔어독일어러시아어몽골어미얀마어(버마어)베트남어스페인어아랍어영어인도네시아어...
13    [관광음식점] [백년가게]돈비어천가 ( 인천광역시 중구 개항로 )\n2023.08....
14    놀러는 모든 여행자의 취향을존중하는 여행 포털서비스입니다.\n          \n...
15    주메뉴\n\n\n전화번호 묻고답하기\n내 비즈니스 등록\n전화번호 제보\n\n이벤트...
16    다후니\n            \n          \n\n\n\n\n메뉴\n\n\...
17                                              

In [28]:
hannanum = Hannanum()
text_list = []
for index, row in df.iterrows():
    row_list = row.tolist()  # 각 행을 리스트로 변환
    row_listlist = row_list.tolist().tolist()
#     print(row_list)
    text_list.append(hannanum.nouns(row_listlist))  # 리스트를 다른 리스트에 추가
#     text_list.append(row_list)
# row_list
text_list

AttributeError: 'list' object has no attribute 'tolist'